# Laboratorio: Aprendizaje Supervisado

### Integrantes:
### Javier Benitez 23405, Javier Lopéz 


pip install scikit-learn

In [14]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing_data = fetch_california_housing(as_frame=True)

df = housing_data.frame
X = housing_data.data
y = housing_data.target

In [ ]:
print(df.describe())

             MedInc      HouseAge      AveRooms     AveBedrms    Population  \
count  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000   
mean       3.870671     28.639486      5.429000      1.096675   1425.476744   
std        1.899822     12.585558      2.474173      0.473911   1132.462122   
min        0.499900      1.000000      0.846154      0.333333      3.000000   
25%        2.563400     18.000000      4.440716      1.006079    787.000000   
50%        3.534800     29.000000      5.229129      1.048780   1166.000000   
75%        4.743250     37.000000      6.052381      1.099526   1725.000000   
max       15.000100     52.000000    141.909091     34.066667  35682.000000   

           AveOccup      Latitude     Longitude   MedHouseVal  
count  20640.000000  20640.000000  20640.000000  20640.000000  
mean       3.070655     35.631861   -119.569704      2.068558  
std       10.386050      2.135952      2.003532      1.153956  
min        0.692308     32.54000

## Breve Analisis

La variable objetivo es MedHouseVal y se puede ver que los datos estan bastante agrupados a que tienen un std cercano a 1


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

print(f"Total de datos: {len(X)}")
print(f"Entrenamiento: {len(X_train)}")
print(f"Prueba: {len(X_test)}")

Total de datos: 20640
Entrenamiento: 16512
Prueba: 4128


## Justificar los valores usados para test_size y random_state.

Se uso el 20% ya que es el valor "optimo" de datos segun la regla de pareto que el 80% de los resultados vienen de el primer 20% y si usamos mas datos como el 50% se estarian desperdiciando datos valiosos que el modelo nececita para aprender.

Y para el random state ya que actua como una "semilla" para la reproducibilidad de los datos.